In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.05)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.75 sec - Loss 0.103927 - ACC 81.09% - ACC Mean 81.09% - AUC 88.62% - AUC Mean 88.62% - Deter 000
Ite 00050 - 0.10 sec - Loss 0.097284 - ACC 83.21% - ACC Mean 81.64% - AUC 88.59% - AUC Mean 88.55% - Deter 022
Ite 00100 - 0.09 sec - Loss 0.080281 - ACC 82.35% - ACC Mean 81.42% - AUC 87.44% - AUC Mean 87.36% - Deter 072
Ite 00150 - 0.10 sec - Loss 0.052228 - ACC 80.78% - ACC Mean 81.82% - AUC 87.35% - AUC Mean 87.83% - Deter 122
Ite 00200 - 0.10 sec - Loss 0.035158 - ACC 82.14% - ACC Mean 83.50% - AUC 90.06% - AUC Mean 90.13% - Deter 009
Ite 00250 - 0.09 sec - Loss 0.030170 - ACC 84.26% - ACC Mean 83.29% - AUC 90.16% - AUC Mean 90.36% - Deter 059
Ite 00300 - 0.09 sec - Loss 0.028655 - ACC 81.54% - ACC Mean 82.61% - AUC 89.78% - AUC Mean 90.19% - Deter 109
Ite 00350 - 0.10 sec - Loss 0.027947 - ACC 81.84% - ACC Mean 82.77% - AUC 90.15% - AUC Mean 90.14% - Deter 159
Ite 00400 - 0.10 sec - Loss 0.027472 - ACC 82.94% - ACC Mean 82.76% - AUC 89.94% - AUC Mean 90.13% - Deter 002
I

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 79.99% +- 0.0% - auc: 89.8% +- 0.0%
MEAN - acc: 85.92% +- 0.0% - auc: 90.72% +- 0.0%
KNN - acc: 85.83% +- 0.0% - auc: 90.9% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:36<00:00, 138.34it/s]



GAIN RUN 1/10 - acc: 81.27% +- 3.55% - auc: 87.56% +- 1.74%



100%|██████████| 5000/5000 [00:35<00:00, 139.16it/s]



GAIN RUN 2/10 - acc: 81.51% +- 4.88% - auc: 88.97% +- 3.02%



100%|██████████| 5000/5000 [00:36<00:00, 138.21it/s]


...


GAIN RUN 8/10 - acc: 84.01% +- 3.7% - auc: 88.35% +- 3.28%



100%|██████████| 5000/5000 [00:39<00:00, 125.51it/s]



GAIN RUN 9/10 - acc: 87.66% +- 1.99% - auc: 92.16% +- 1.42%



100%|██████████| 5000/5000 [00:39<00:00, 126.06it/s]



GAIN RUN 10/10 - acc: 84.41% +- 2.06% - auc: 90.28% +- 2.14%

GAIN GLOBAL - acc: 83.66% +- 2.46% - auc: 89.35% +- 1.81%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 85.22% +- 0.0% - auc: 90.45% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 1/10 - acc: 85.5% +- 1.81% - auc: 90.85% +- 1.33%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOREST RUN 2/10 - acc: 84.0% +- 3.21% - auc: 90.73% +- 1.76%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 3/10 - acc: 82.0% +- 2.39% - auc: 90.36% +- 1.12%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 4/10 - acc: 84.9% +- 1.52% - auc: 90.66% +- 1.57%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 5/10 - acc: 82.8% +- 2.57% - auc: 90.92% +- 1.69%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 6/10 - acc: 83.45% +- 3.57% - auc: 90.96% +- 1.81%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 86.3% +- 2.98% - auc: 90.29% +- 2.2%
SOFTIMPUTE RUN 2/10 - acc: 84.48% +- 1.04% - auc: 91.18% +- 2.52%
SOFTIMPUTE RUN 3/10 - acc: 81.47% +- 7.71% - auc: 90.14% +- 3.89%
SOFTIMPUTE RUN 4/10 - acc: 84.6% +- 4.41% - auc: 89.93% +- 3.59%
SOFTIMPUTE RUN 5/10 - acc: 87.63% +- 2.74% - auc: 91.74% +- 1.87%
SOFTIMPUTE RUN 6/10 - acc: 84.27% +- 2.42% - auc: 91.47% +- 0.71%
SOFTIMPUTE RUN 7/10 - acc: 85.92% +- 2.9% - auc: 90.75% +- 2.9%
SOFTIMPUTE RUN 8/10 - acc: 84.36% +- 8.86% - auc: 89.64% +- 4.99%
SOFTIMPUTE RUN 9/10 - acc: 84.36% +- 7.56% - auc: 89.92% +- 4.04%
SOFTIMPUTE RUN 10/10 - acc: 84.27% +- 5.53% - auc: 89.63% +- 3.83%

SOFTIMPUTE GLOBAL - acc: 84.76% +- 1.54% - auc: 90.47% +- 0.73%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.8793

SINKHORN RUN 1/10 - acc: 80.46% +- 5.63% - auc: 89.12% +- 3.75%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.7879

SINKHORN RUN 2/10 - acc: 82.9% +- 1.76% - auc: 90.7% +- 2.43%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.8190

SINKHORN RUN 3/10 - acc: 80.53% +- 1.76% - auc: 88.39% +- 1.68%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.8065

SINKHORN RUN 4/10 - acc: 86.73% +- 1.48% - auc: 91.71% +- 1.2%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.8547

SINKHORN RUN 5/10 - acc: 84.19% +- 3.84% - auc: 88.97% +- 2.06%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.8141

SINKHORN RUN 6/10 - acc: 78.52% +- 2.53% - auc: 87.82% +- 2.57%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.7769

SINKHORN RUN 7/10 - acc: 82.23% +- 2.49% - auc: 89.89% +- 1.82%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 0.7720

SINKHORN RUN 8/10 - acc: 84.89% +- 3.68% - auc: 90.3

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 82.05% +- 4.94% - auc: 89.52% +- 2.39%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 84.71% +- 1.46% - auc: 89.91% +- 1.52%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 82.42% +- 4.32% - auc: 89.52% +- 2.59%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 80.91% +- 4.93% - auc: 89.41% +- 3.83%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 83.68% +- 3.75% - auc: 90.62% +- 2.59%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 82.38% +- 5.34% - auc: 89.47% +- 2.83%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 82.5% +- 1.06% - auc: 89.25% +- 2.19%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 84.1% +- 2.51% - auc: 90.37% +- 1.97%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is even with MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is even with KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is even with SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly better than MIDA
